In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import ConfusionMatrixDisplay

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Flatten, Conv2D, MaxPooling2D,LeakyReLU
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import mnist, cifar10
from tensorflow import keras

In [2]:
data_test = pd.read_csv('data/fashion-mnist_test.csv')
df_test = pd.DataFrame(data_test)

FileNotFoundError: [Errno 2] No such file or directory: 'data/fashion-mnist_test.csv'

In [ ]:
data_train = pd.read_csv('data/fashion-mnist_train.csv')
df_train = pd.DataFrame(data_test)

### Setting X & y

In [ ]:
X = df_train.drop('label',axis=1)
y = df_train['label']

In [ ]:
y.value_counts().count()

### Normalize Data

In [ ]:
X = X/255

### Split X and y into train and test subsets.


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=.2,random_state=2023)

In [ ]:
X_train.shape , y_train.shape

In [ ]:
X_train_cnn = X_train.values.reshape(-1,28,28,1)
X_val_cnn = X_val.values.reshape(-1,28,28,1)

In [ ]:
X_train_cnn.shape[1:]

### Create Network

#### Convolutional NN

In [ ]:
model = Sequential([
    Conv2D(64,(3,3), activation=LeakyReLU(alpha=0.05), input_shape = (X_train_cnn.shape[1:])),
    MaxPooling2D(2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.6),
    Dense(10,activation = 'softmax') # there are 10 unique values in y
])  

### Train your network

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy', 
    metrics = ['accuracy'],
)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=6)

In [ ]:
history = model.fit(
    X_train_cnn,
    y_train,
    validation_data=(X_val_cnn, y_val),
    batch_size = 32,
    epochs = 15,
    verbose= 1,
    callbacks=[early_stopping]
)

In [ ]:
plt.plot(history.epoch, history.history['loss'], c='r',label = 'Training Loss');    
plt.plot(history.epoch, history.history['val_loss'], c='pink', label = 'Validation Loss');
plt.yscale('linear')

plt.title('Training & Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend();

### Running Model on All Data

In [ ]:
X_cnn = X.values.reshape(-1,28,28,1)


In [ ]:
model_all = Sequential([
    Conv2D(64,(3,3), activation=LeakyReLU(alpha=0.05), input_shape = (X_cnn.shape[1:])),
    MaxPooling2D(2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.6),
    Dense(10,activation = 'softmax') # there are 10 unique values in y
])  

In [ ]:
model_all.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy', 
    metrics = ['accuracy'],
)

In [ ]:
history_all = model_all.fit(
    X_cnn,
    y,
    batch_size = 32,
    epochs = 15,
    verbose= 1,
    callbacks=[early_stopping]
)

#### Making Predictions

In [ ]:
X_test = df_test.drop('label',axis =1)

In [ ]:
X_test_cnn = X_test.values.reshape(-1,28,28,1)

In [ ]:
predictions = model_all.predict(X_test_cnn)

In [ ]:
X_test['Predictions'] = np.argmax(predictions,axis=1)

In [ ]:
X_test.to_csv('data/submission.csv',index= False)

### Visualizations

In [ ]:
val_preds = np.argmax(model.predict(X_val_cnn),axis=1)
val_preds.shape

In [ ]:
val_preds.shape

In [ ]:
val_preds

In [ ]:
val_preds.shape

In [ ]:
X_val_cnn.shape

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_val, val_preds, cmap = 'Reds');
plt.title("Confusion Matrix")
plt.savefig('images/conf_matrix.png')

In [ ]:
y_train.label[12]

In [ ]:
plt.imshow(X_train_cnn[12],cmap = 'binary')
plt.title('Blouse')

In [ ]:
y_val.reset_index().drop('index',axis=1)

In [ ]:
y_val = np.array(y_val)

In [ ]:
wrong_ix = np.where(val_preds != y_val)[0]
for i in wrong_ix[:5]:
    plt.clf()
    plt.imshow(X_val_cnn[i], cmap='binary')
    plt.show()
    pred = val_preds[i]
    truth = y_val[i]
    print(f'We thought = {pred}, but really {truth}')